In [ ]:
from pathlib import Path

import shapefile
import shapely.geometry as sgeom
from shapely.ops import unary_union
import numpy as np

from amap import (
    records_to_df,
    shorten_ct_name,
    shorten_pr_name,
    polygon_to_polys,
    make_prj_file
)

实际上这里的 `cn_city.shp` 已经是用 mapshaper 简化后的版本了。再从市级数据制作省级和国级数据，能保证三个级别的边界相同。

In [ ]:
data_dirpath = Path('../data')

ct_shp_filepath = data_dirpath / 'cn_city.shp'
ct_csv_filepath = data_dirpath / 'cn_city.csv'

pr_shp_filepath = data_dirpath / 'cn_province.shp'
pr_prj_filepath = data_dirpath / 'cn_province.prj'
pr_csv_filepath = data_dirpath / 'cn_province.csv'

bd_shp_filepath = data_dirpath / 'cn_border.shp'
bd_prj_filepath = data_dirpath / 'cn_border.prj'

制作市级数据的属性表，字段有：

- `ct_name`
- `ct_adcode`
- `pr_name`
- `pr_adcode`
- `abbr_name`：`ct_name` 的缩写。
- `lon`：代表点的经度。
- `lat`：代表点的纬度。

代表点仅供参考，后续可以手工修改。

In [ ]:
df_ct = records_to_df(ct_shp_filepath)
df_ct['short_name'] = list(map(shorten_ct_name, df_ct['ct_name']))

with shapefile.Reader(str(ct_shp_filepath)) as reader:
    polygons = list(map(sgeom.shape, reader.shapes()))

lons, lats = [], []
for polygon in polygons:
    point = polygon.representative_point()
    lons.append(point.x)
    lats.append(point.y)
df_ct['lon'] = np.round(lons, 1)
df_ct['lat'] = np.round(lats, 1)

df_ct.to_csv(str(ct_csv_filepath), index=False)
df_ct.head(10)

将同属一个省的市级多边形合并成省级多边形，从而制作省级数据。同时制作配套的属性表，字段有：

- `pr_name`
- `pr_adcode`
- `abbr_name`：`pr_name` 的缩写。
- `lon`：代表点的经度。
- `lat`：代表点的纬度。

In [ ]:
df_pr = (df_ct
    [['pr_name', 'pr_adcode']]
    .drop_duplicates('pr_name')
    .reset_index(drop=True)
)

with shapefile.Reader(str(ct_shp_filepath)) as reader:
    shapeRecs = reader.shapeRecords()

lons, lats = [], []
with shapefile.Writer(str(pr_shp_filepath), shapeType=5) as writer:
    writer.fields = [['pr_name', 'C', 80, 0], ['pr_adcode', 'N', 6, 0]]
    for record in df_pr.itertuples():
        polygons = []
        for shapeRec in shapeRecs:
            if shapeRec.record.pr_name == record.pr_name:
                polygons.append(sgeom.shape(shapeRec.shape))
        polygon = unary_union(polygons)
        polys = polygon_to_polys(polygon)
        writer.record(**record._asdict())
        writer.poly(polys)

        point = polygon.representative_point()
        lons.append(point.x)
        lats.append(point.y)
        print(record)

make_prj_file(pr_prj_filepath)
df_pr['short_name'] = list(map(shorten_pr_name, df_pr['pr_name']))
df_pr['lon'] = np.round(lons, 1)
df_pr['lat'] = np.round(lats, 1)
df_pr.to_csv(str(pr_csv_filepath), index=False)
df_pr.head(10)

将省级多边形合并制作国界数据。

In [ ]:
with shapefile.Reader(str(pr_shp_filepath)) as reader:
    polygons = list(map(sgeom.shape, reader.shapes()))
polygon = unary_union(polygons)
polys = polygon_to_polys(polygon)

make_prj_file(bd_prj_filepath)
with shapefile.Writer(str(bd_shp_filepath), shapeType=5) as writer:
    writer.fields = [['cn_name', 'C', 80, 0], ['cn_adcode', 'N', 6, 0]]
    writer.record(cn_name='中华人民共和国', cn_adcode=100000)
    writer.poly(polys)